# Output_2_Excel

## send dataframe(s) to excel

In [1]:
import pandas as pd
#import openpyxl

In [ ]:
def df_to_xl(df, output_file, output_tab, start_row=0, start_column=0, header=False):
    '''
    This function opens the workbook output_file, reads in all of the sheets to that file and sends
    the dataframe df to a location on the output_tab sheet. The location is determined by the start_row and
    start_column sent to the function.
    
    Inputs
        df - the dataframe that will be sent to excel
        output_file - the path and filename of the output file
        output_tab - the tab in the output file to send the data to.
        start_row - the first row where the data will appear. NOTE: Python treats cell A1 as row zero.
        start_column - the first column where the data will appear. NOTE: Python treats cell A1 as column zero.
        header - if the header is True, it will send the column names of the dataframe to excel. 
                If header is false, no column names will be sent to the excel file.
                If header is set equal to a list of strings, the list of strings will be an alias to the column names and will
                    be sent to the excel file and the column names will not.
        
    Python Modules to be imported for this to work
        pandas
        openpyxl
    '''
    
    #Importing openpyxl module
    import openpyxl
    
    # Open the output file using openpyxl module
    book = openpyxl.load_workbook(output_file)

    # creating variable 'writer' that reads in all of the data in the output_file spreadsheet
    # Loading the spreadsheet with openpyxl allows python to write the spreadsheet back to the file with new data
    writer = pd.ExcelWriter(output_file, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    # sending the dataframe 'df' to the 'writer' variable and specifying where this dataframe should go in the spreadsheet
    df.to_excel(writer, 
                sheet_name=output_tab, 
                startrow = start_row, 
                startcol = start_col, 
                header = header, 
                index=False)

    writer.save()

    writer.close()

    print('DataFrame is written successfully to Excel File.')
    
    return None


In [ ]:
#df_to_xl(df, path, output_tab, start_row, start_col)

## Sending multiple dataframes to excel

In [2]:
def multiple_dfs_to_xl(output_dict, template_path_and_file, save_path_and_file=False, time_stamp_format=False):
    '''
    This function copies the 'template_file' excel workbook, renames the workbook (with a timestamp into the same directory) and 
    opens the renamed copy of the workbook 'template_file'.   Then it reads in all of the sheets of the opened workbook and sends multiple
    dataframes to multiple tabs at specific locations on the the excel sheet. The 'tab' fields in the dictionaries determine which
    worksheet the data gets sent to and the location within that worksheet is determined by the start_row and
    start_column contained in the dictionaries.
    
    Inputs
        output_dict - a dictionary comprised of dictionaries. The keys of the outter-most dictionary are the 
        
    
        output_dict = {'data_1': {'var_name': df,
                              'tab': 'first_page',
                              'start_row': 7,
                              'start_col': 1,
                              'header': False,
                              'index': False},
                  'data_2': {'var_name': df,
                             'tab': 'first_page',
                             'start_row': 7,
                             'start_col': 1,
                             'header': False,
                             'index': False}
                  }

            The inner dictionaries are comprised of the following keys
        
            df - the dataframe that will be sent to excel
            template_file - the path and filename of the excel workbook template the dataframes will be written to
            output_tab - the tab in the output file to send the data to.
            start_row - the first row where the data will appear. NOTE: Python treats cell A1 as row zero.
            start_column - the first column where the data will appear. NOTE: Python treats cell A1 as row zero.
            header - if the header is True, it will send in the column names of the dataframe. 
                If header is false, no column names will be sent to the excel file.
                If header contains a list of strings, the list of strings will be an alias to the column names
            index - boolean variable that determines whether the index of the dataframe will be sent to excel.
        
        template_path_and_file - an excel workbook (including that path to the workbook) that contains the 
            formatted worksheets that will recieve the dataframes from python
        
        save_path_and_file - Optional - if provided it can be a path only (ex.. '../data/') 
            or path and outfile name (ex.. (ex.. '../data/sample_output_file_name.xlsx') ) if omitted the 
            output file will retain the same name as the template with a timestamp appended to the filename saved
            in the same directory as the template.  
            Timestamp will append to filename if no filename is passed.  
            Timestamp will also append to filename if a save_path_and_file filename is provided unless the 'omit' 
                    Timestamp value is included in the function call. 
            To be clear, by using the timestamp it is the goal of this function to prevent overwriting the excel template.  
        
        time_stamp_format - Optional - 
            Acceptable values are as follows:
             * 'hr' - (ex.. 22Jul2019)
             * 'min' - (ex.. 22Jul201901)
             * 'sec' - (ex.. 22Jul20190127)
             * 'def_ts' - to explicitly use the default timestamp format (ex.. 22Jul20) 
             * 'omit' - to explicitly not add a timestamp to the output file name
            If the time_stamp_format is not supplied or any other value that is not defined above is used
            the default timestamp (ex..22Jul20) will be added to the output file name.
             
             
    Python Modules to be imported for this to work
        pandas
        openpyxl
        shutil
        os
        '''
    
    import openpyxl
    import datetime
    import shutil
    import os
    
    #troubleshooting tips
    tips = '''Troubleshooting tips:
        >Filenames must end with '.xlsx'
        >Check spelling 
        >Use forward slashes /'s in your path
        >Make sure you can access the file and/or path from your computer (Is it on your harddisk or the network(VPN)?
        >Call the Python Hotline (408)536.2086'''
    
    
#Process Excel Template path and filename    
    #check to make sure the template path and file exists as specified  
    if os.path.isfile(template_path_and_file):
        if template_path_and_file.rfind('.xlsx') > 0:
                #yes '.xlsx' was found.  Therefore a valid filename is in the template_path_and_file string 
                txt = "Template confirmed {directory} "
                print(txt.format(directory = template_path_and_file))
        else:
            #no '.xlsx' found so there isn't a valid filename included in save_path_and_file string
            #print error message and exit function
            txt = "Error: template_path_and_file supplied '{directory}' is invalid."
            print(txt.format(directory = template_path_and_file))
            print(tips)
            return
            
    else:
        txt = "Error: template_path_and_file '{directory}' doesn't exist."
        print(txt.format(directory = template_path_and_file))
        print(tips)
        return

    # create a dictionary for the template path and filename
    template_path_and_file_dict = {'file_path':False,
                                  'file_name': False,
                                  'time_stamp': False,
                                  'file_name_suffix': False}
    #find the index where the path ends and the filename begins
    file_path_index = template_path_and_file.rfind('/')+1
    #update dictionary with the path
    template_path_and_file_dict['file_path'] = template_path_and_file[:file_path_index] 

    #find the index where the filename extension begins
    suffix_index = template_path_and_file.rfind('.')
    #update dictionary with the path
    template_path_and_file_dict['file_name_suffix'] = template_path_and_file[suffix_index:]
    #update dictionary with filename
    template_path_and_file_dict['file_name'] = template_path_and_file[file_path_index:suffix_index]
    
#Process output path and filename 
    # create a dictionary for the save path and filename
    save_path_and_file_dict = {'file_path':False,
                              'file_name': False,
                              'time_stamp': False,
                              'file_name_suffix': False}

    save_number = 0
    # is there is a value for the save_path_and_file
    if not save_path_and_file == False:
        #yes there is a save_path_and_file
        #is check for proper filename by confirming '.xlsx' is the last 5 characters in the string 
        if save_path_and_file[-5:] == '.xlsx':
            #yes proper filename, add 1 to save_number
            save_number += 1
        # is there a slash to indicate a path is part of the save_path_and_file supplied     
        if save_path_and_file.rfind('/') > 0: 
            #yes there is a path, add 2 to save_number
            save_number += 2
        
        #save_path_and_file is a filename only, put filename in dictionary and copy template path to dictionary
        if save_number == 1:
            save_path_and_file_dict['file_path'] =  template_path_and_file_dict['file_path']
            save_suffix_index = save_path_and_file.rfind('.')
            save_path_and_file_dict['file_name_suffix'] = save_path_and_file[save_suffix_index:]
            save_path_and_file_dict['file_name'] = save_path_and_file[:save_suffix_index]

        #save_path_and_file is a path only, put path in dictionary and copy template filename and suffix to dictionary
        elif save_number == 2:
            #is the end of the path a /?
            if save_path_and_file[-1] == '/':
                #yes the end of the path is a /, no changes needed add path to dictionary as is. 
                save_path_and_file_dict['file_path'] = save_path_and_file
            else:
                #no the end of the path isn't a /, add a / at the end of the path and add modified path to the dictionary
                save_path_and_file_dict['file_path'] = save_path_and_file + '/'
            save_path_and_file_dict['file_name'] = template_path_and_file_dict['file_name']
            save_path_and_file_dict['file_name_suffix'] = template_path_and_file_dict['file_name_suffix']

        #save_path_and_file is a path and file name, put path, filename and suffix in dictionary
        elif save_number == 3:
            #locate the end of the path for the output 
            save_path_index = save_path_and_file.rfind('/')+1
            save_path_and_file_dict['file_path'] = save_path_and_file[:save_path_index]
            save_suffix_index = save_path_and_file.rfind('.')
            save_path_and_file_dict['file_name_suffix'] = save_path_and_file[save_suffix_index:]
            save_path_and_file_dict['file_name'] = save_path_and_file[save_path_index:save_suffix_index]
             
        else:
            txt = "Error: unable to process save_path_and_file, '{directory}' "
            print(txt.format(directory = save_path_and_file))
            print(tips)
            return

    else:
        #no save_path_and_file input, use template path, filename and suffix for output file
        save_path_and_file_dict = template_path_and_file_dict
            
    #check to make sure the output path exists as specified  
    if os.path.isdir(save_path_and_file_dict['file_path']):
        txt = "Save directory confirmed '{directory}'"
        print(txt.format(directory = save_path_and_file_dict['file_path']))
    else:
        txt = "Error: save_path_and_file, '{directory}', is not valid."
        print(txt.format(directory= save_path_and_file))
        print(tips)
        return                  
    
#Process TimeStamp 
    #create timestamp format options dictionary 
    timestamp_dictionary = {'hr': '%d''%b''%y''%H',
                           'min': '%d''%b''%y''%H''%M',
                           'sec': '%d''%b''%y''%H''%M''%S' ,
                           'def_ts': '%d''%b''%y',
                           'omit':''}
    #has a value been supplied for the time_stamp_format option?
    if time_stamp_format:
            #yes time_stamp_format supplied
            #is the format found in the timestamp_dictionary?
            if time_stamp_format in timestamp_dictionary:
                #yes format found, use corresponding format from dictionary
                time_stamp_string = timestamp_dictionary[time_stamp_format]
            else:
                #no format not found, use default timestamp format from dictionary
                time_stamp_format = 'def_ts'
                time_stamp_string = timestamp_dictionary[time_stamp_format]
    else:
        #no time_stamp_format not supplied, use default timestamp format from dictionary  
        time_stamp_format = 'def_ts'
        time_stamp_string = timestamp_dictionary[time_stamp_format]
    #update save_path_and_file_dict with outcome of timestamp routine along with a '_'
    save_path_and_file_dict['time_stamp'] = '_' +  datetime.datetime.now().strftime(time_stamp_string)
    #will the template details be used for the output file?
    if not (save_path_and_file_dict['file_path'] == template_path_and_file_dict[
        'file_path'] and save_path_and_file_dict['file_name'] == template_path_and_file_dict['file_name']):
        #no output file and path will not be the same as the template
        #is timestamp only the '-' character?
        if save_path_and_file_dict['time_stamp'] == '_':
            #yes, since path and filename are different from the template the '_' is not necessary and is removed.
            save_path_and_file_dict['time_stamp'] = ''
        



#create save file output path and name 
    #initialize path and filename variable
    new_file_name = None
    #loop through the save_path_and_file_dict to build the output path and filename 
    for items in save_path_and_file_dict:
        #ignore save_path_and_file_dict items if they are set to false, like when a timestamp is 'omit'ted. 
        if not save_path_and_file_dict[items] == False:
            #is this the first item in the dictionary being processed?
            if new_file_name:
                #no, so concatenate existing filename with next item in the dictionary
                new_file_name = new_file_name + save_path_and_file_dict[items]
            else:
                #yes, first time through so set the path and filename variable equal to the first dictionary item(path)
                new_file_name = save_path_and_file_dict[items]
        #give concatenated path and file name a new variable to disambiguate        
        new_copied_file_name = new_file_name  
    txt = "Template successfully saved to new file: '{directory}'"
    print(txt.format(directory= new_copied_file_name))

    
#copy template and save with new concatenated name using shutil module
    output_file = shutil.copy(template_path_and_file, new_copied_file_name) 

#Output Dataframes to Workbook    
    #Using openpyxl module to open workbook to write to, discover tabs in workbook
    book = openpyxl.load_workbook(output_file)
    writer = pd.ExcelWriter(output_file, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    #iterate through dictionary and output dataframes to worksheets
    for key in output_dict.keys():

        output_dict[key]['var_name'].to_excel(writer, 
                                              sheet_name = output_dict[key]['tab'], 
                                              startrow = output_dict[key]['start_row'], 
                                              startcol = output_dict[key]['start_col'], 
                                              header = output_dict[key]['header'], 
                                              index = output_dict[key]['index'])

    writer.save()
    writer.close()
    txt = "DataFrame written successfully to: '{directory}'"
    print(txt.format(directory= new_copied_file_name))
    print('\n \n')

    return None

In [86]:
def multiple_dfs_to_xl(output_dict, template_path_and_file, save_path_and_file=False, time_stamp_format=False):
    '''
    This function copies the 'template_file' excel workbook, renames the workbook (with a timestamp into the same directory) and 
    opens the renamed copy of the workbook 'template_file'.   Then it reads in all of the sheets of the opened workbook and sends multiple
    dataframes to multiple tabs at specific locations on the the excel sheet. The 'tab' fields in the dictionaries determine which
    worksheet the data gets sent to and the location within that worksheet is determined by the start_row and
    start_column contained in the dictionaries.
    
    Inputs
        output_dict - a dictionary comprised of dictionaries. The keys of the outter-most dictionary are the 
        
    
        output_dict = {'data_1': {'var_name': df,
                              'tab': 'first_page',
                              'start_row': 7,
                              'start_col': 1,
                              'header': False,
                              'index': False},
                  'data_2': {'var_name': df,
                             'tab': 'first_page',
                             'start_row': 7,
                             'start_col': 1,
                             'header': False,
                             'index': False}
                  }

            The inner dictionaries are comprised of the following keys
        
            df - the dataframe that will be sent to excel
            template_file - the path and filename of the excel workbook template the dataframes will be written to
            output_tab - the tab in the output file to send the data to.
            start_row - the first row where the data will appear. NOTE: Python treats cell A1 as row zero.
            start_column - the first column where the data will appear. NOTE: Python treats cell A1 as row zero.
            header - if the header is True, it will send in the column names of the dataframe. 
                If header is false, no column names will be sent to the excel file.
                If header contains a list of strings, the list of strings will be an alias to the column names
            index - boolean variable that determines whether the index of the dataframe will be sent to excel.
        
        template_path_and_file - an excel workbook (including that path to the workbook) that contains the 
            formatted worksheets that will recieve the dataframes from python
        
        save_path_and_file - Optional - if provided it can be a path only (ex.. '../data/') 
            or path and outfile name (ex.. (ex.. '../data/sample_output_file_name.xlsx') ) if omitted the 
            output file will retain the same name as the template with a timestamp appended to the filename saved
            in the same directory as the template.  
            Timestamp will append to filename if no filename is passed.  
            Timestamp will also append to filename if a save_path_and_file filename is provided unless the 'omit' 
                    Timestamp value is included in the function call. 
            To be clear, by using the timestamp it is the goal of this function to prevent overwriting the excel template.  
        
        time_stamp_format - Optional - 
            Acceptable values are as follows:
             * 'hr' - (ex.. 22Jul2019)
             * 'min' - (ex.. 22Jul201901)
             * 'sec' - (ex.. 22Jul20190127)
             * 'def_ts' - to explicitly use the default timestamp format (ex.. 22Jul20) 
             * 'omit' - to explicitly not add a timestamp to the output file name
            If the time_stamp_format is not supplied or any other value that is not defined above is used
            the default timestamp (ex..22Jul20) will be added to the output file name.
             
             
    Python Modules to be imported for this to work
        pandas
        openpyxl
        shutil
        os
        '''
    
    import openpyxl
    import datetime
    import shutil
    import os
    
    #troubleshooting tips
    tips = '''Troubleshooting tips:
        >Filenames must end with '.xlsx'
        >Check spelling 
        >Use forward slashes /'s in your path
        >Make sure you can access the file and/or path from your computer (Is it on your harddisk or the network(VPN)?
        >Call the Python Hotline (408)536.2086'''
    
    
    #Process Excel Template path and filename    
    #check to make sure the template path and file exists as specified  
    if os.path.isfile(template_path_and_file):
        if template_path_and_file.rfind('.xlsx') > 0:
                #yes '.xlsx' was found.  Therefore a valid filename is in the template_path_and_file string 
                txt = "Template confirmed {directory} "
                print(txt.format(directory = template_path_and_file))
        else:
            #no '.xlsx' found so there isn't a valid filename included in save_path_and_file string
            #print error message and exit function
            txt = "Error: template_path_and_file supplied '{directory}' is invalid."
            print(txt.format(directory = template_path_and_file))
            print(tips)
            return
            
    else:
        txt = "Error: template_path_and_file '{directory}' doesn't exist."
        print(txt.format(directory = template_path_and_file))
        print(tips)
        return

    # create a dictionary for the template path and filename
    template_path_and_file_dict = {'file_path':False,
                                  'file_name': False,
                                  'time_stamp': False,
                                  'file_name_suffix': False}
    #find the index where the path ends and the filename begins
    file_path_index = template_path_and_file.rfind('/')+1
    #update dictionary with the path
    template_path_and_file_dict['file_path'] = template_path_and_file[:file_path_index] 

    #find the index where the filename extension begins
    suffix_index = template_path_and_file.rfind('.')
    #update dictionary with the path
    template_path_and_file_dict['file_name_suffix'] = template_path_and_file[suffix_index:]
    #update dictionary with filename
    template_path_and_file_dict['file_name'] = template_path_and_file[file_path_index:suffix_index]
    
    #Process output path and filename 
    # create a dictionary for the save path and filename
    save_path_and_file_dict = {'file_path':False,
                              'file_name': False,
                              'time_stamp': False,
                              'file_name_suffix': False}

    save_number = 0
    # is there is a value for the save_path_and_file
    if not save_path_and_file == False:
        #yes there is a save_path_and_file
        #is check for proper filename by confirming '.xlsx' is the last 5 characters in the string 
        if save_path_and_file[-5:] == '.xlsx':
            #yes proper filename, add 1 to save_number
            save_number += 1
        # is there a slash to indicate a path is part of the save_path_and_file supplied     
        if save_path_and_file.rfind('/') > 0: 
            #yes there is a path, add 2 to save_number
            save_number += 2
        
        #save_path_and_file is a filename only, put filename in dictionary and copy template path to dictionary
        if save_number == 1:
            save_path_and_file_dict['file_path'] =  template_path_and_file_dict['file_path']
            save_suffix_index = save_path_and_file.rfind('.')
            save_path_and_file_dict['file_name_suffix'] = save_path_and_file[save_suffix_index:]
            save_path_and_file_dict['file_name'] = save_path_and_file[:save_suffix_index]

        #save_path_and_file is a path only, put path in dictionary and copy template filename and suffix to dictionary
        elif save_number == 2:
            #is the end of the path a /?
            if save_path_and_file[-1] == '/':
                #yes the end of the path is a /, no changes needed add path to dictionary as is. 
                save_path_and_file_dict['file_path'] = save_path_and_file
            else:
                #no the end of the path isn't a /, add a / at the end of the path and add modified path to the dictionary
                save_path_and_file_dict['file_path'] = save_path_and_file + '/'
            save_path_and_file_dict['file_name'] = template_path_and_file_dict['file_name']
            save_path_and_file_dict['file_name_suffix'] = template_path_and_file_dict['file_name_suffix']

        #save_path_and_file is a path and file name, put path, filename and suffix in dictionary
        elif save_number == 3:
            #locate the end of the path for the output 
            save_path_index = save_path_and_file.rfind('/')+1
            save_path_and_file_dict['file_path'] = save_path_and_file[:save_path_index]
            save_suffix_index = save_path_and_file.rfind('.')
            save_path_and_file_dict['file_name_suffix'] = save_path_and_file[save_suffix_index:]
            save_path_and_file_dict['file_name'] = save_path_and_file[save_path_index:save_suffix_index]
             
        else:
            txt = "Error: unable to process save_path_and_file, '{directory}' "
            print(txt.format(directory = save_path_and_file))
            print(tips)
            return

    else:
        #no save_path_and_file input, use template path, filename and suffix for output file
        save_path_and_file_dict = template_path_and_file_dict
            
    #check to make sure the output path exists as specified  
    if os.path.isdir(save_path_and_file_dict['file_path']):
        txt = "Save directory confirmed '{directory}'"
        print(txt.format(directory = save_path_and_file_dict['file_path']))
    else:
        txt = "Error: save_path_and_file, '{directory}', is not valid."
        print(txt.format(directory= save_path_and_file))
        print(tips)
        return                  
    
    #Process TimeStamp 
    #create timestamp format options dictionary 
    timestamp_dictionary = {'hr': '%d''%b''%y''%H',
                           'min': '%d''%b''%y''%H''%M',
                           'sec': '%d''%b''%y''%H''%M''%S' ,
                           'def_ts': '%d''%b''%y',
                           'omit':''}
    #has a value been supplied for the time_stamp_format option?
    if time_stamp_format:
            #yes time_stamp_format supplied
            #is the format found in the timestamp_dictionary?
            if time_stamp_format in timestamp_dictionary:
                #yes format found, use corresponding format from dictionary
                time_stamp_string = timestamp_dictionary[time_stamp_format]
            else:
                #no format not found, use default timestamp format from dictionary
                time_stamp_format = 'def_ts'
                time_stamp_string = timestamp_dictionary[time_stamp_format]
    else:
        #no time_stamp_format not supplied, use default timestamp format from dictionary  
        time_stamp_format = 'def_ts'
        time_stamp_string = timestamp_dictionary[time_stamp_format]
    #update save_path_and_file_dict with outcome of timestamp routine along with a '_'
    save_path_and_file_dict['time_stamp'] = '_' +  datetime.datetime.now().strftime(time_stamp_string)
    #will the template details be used for the output file?
    if not (save_path_and_file_dict['file_path'] == template_path_and_file_dict[
        'file_path'] and save_path_and_file_dict['file_name'] == template_path_and_file_dict['file_name']):
        #no output file and path will not be the same as the template
        #is timestamp only the '-' character?
        if save_path_and_file_dict['time_stamp'] == '_':
            #yes, since path and filename are different from the template the '_' is not necessary and is removed.
            save_path_and_file_dict['time_stamp'] = ''
        



    #create save file output path and name 
    #initialize path and filename variable
    new_file_name = None
    #loop through the save_path_and_file_dict to build the output path and filename 
    for items in save_path_and_file_dict:
        #ignore save_path_and_file_dict items if they are set to false, like when a timestamp is 'omit'ted. 
        if not save_path_and_file_dict[items] == False:
            #is this the first item in the dictionary being processed?
            if new_file_name:
                #no, so concatenate existing filename with next item in the dictionary
                new_file_name = new_file_name + save_path_and_file_dict[items]
            else:
                #yes, first time through so set the path and filename variable equal to the first dictionary item(path)
                new_file_name = save_path_and_file_dict[items]
        #give concatenated path and file name a new variable to disambiguate        
        new_copied_file_name = new_file_name  
    txt = "Template successfully saved to new file: '{directory}'"
    print(txt.format(directory= new_copied_file_name))

    
    #copy template and save with new concatenated name using shutil module
    output_file = shutil.copy(template_path_and_file, new_copied_file_name) 

    #Output Dataframes to Workbook    
    #Using openpyxl module to open workbook to write to, discover tabs in workbook
    book = openpyxl.load_workbook(output_file)
    writer = pd.ExcelWriter(output_file, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    #iterate through dictionary and output dataframes to worksheets
    for key in output_dict.keys():

        output_dict[key]['var_name'].to_excel(writer, 
                                              sheet_name = output_dict[key]['tab'], 
                                              startrow = output_dict[key]['start_row'], 
                                              startcol = output_dict[key]['start_col'], 
                                              header = output_dict[key]['header'], 
                                              index = output_dict[key]['index'])

    writer.save()
    writer.close()
    txt = "DataFrame written successfully to: '{directory}'"
    print(txt.format(directory= new_copied_file_name))
    print('\n \n')

    return None

# Test code starts here

In [3]:
df = pd.read_excel('../data/sampledfinput.xlsx', sheet_name='sampledata')
df2 = df


In [4]:
output_dict1 = {'var_name': df,
               'tab': 'first_page',
               'start_row': 7,
               'start_col': 1,
               'header': False,
               'index': False}

output_dict2 = {'var_name': df2,
               'tab': 'second_page',
               'start_row': 7,
               'start_col': 0,
               'header': True,
               'index': True}

output_dict = {'data_1': output_dict1,
              'data_2': output_dict2}


In [7]:
template_path_and_file = '../data/sampOutput.xlsx'
save_path_and_file = '../data/sample/sampOutput2.xlsx'
multiple_dfs_to_xl(output_dict, template_path_and_file, save_path_and_file=False, time_stamp_format=False)

Template confirmed ../data/sampOutput.xlsx 
Save directory confirmed '../data/'
Template successfully saved to new file: '../data/sampOutput_28Jul20.xlsx'
DataFrame written successfully to: '../data/sampOutput_28Jul20.xlsx'

 



In [5]:
#Test Routine
input_strings = [False,'../data/sampOutput2.xls', '../data/sampOutput.xlsx', '../data/sampOutput55.xlsx']
output_strings = [False,'..sampled', '../data', '../data/', 'sampOutput2.xlsx', '../data/sample/sampOutput2.xlsx',
                  '../data/sample/sampOutput2.xls']
timestamps = [False,'hr', 'min', 'sec', 'def_ts', 'omit','crunge'] 

j=0
for inps in input_strings:
    j += 1
    k = 0
    print('\n \n \n \n \n')
    for ops in output_strings:
        k += 1
        m = 1
        for stamps in timestamps:
            txt = "(Test input - {inpot}, {outpot}, {tspot})"
            print(txt.format(inpot = j, outpot = k, tspot = m))
            txt = "(Test input - {inpot}, {outpot}, {tspot})\n"
            print(txt.format(inpot = inps, outpot = ops, tspot = stamps))   
            m += 1
            multiple_dfs_to_xl(output_dict, inps, ops, stamps)  
            print('\n\n')

           
              


 
 
 
 

(Test input - 1, 1, 1)
(Test input - False, False, False)

Error: template_path_and_file 'False' doesn't exist.
Troubleshooting tips:
        >Filenames must end with '.xlsx'
        >Check spelling 
        >Use forward slashes /'s in your path
        >Make sure you can access the file and/or path from your computer (Is it on your harddisk or the network(VPN)?
        >Call the Python Hotline (408)536.2086



(Test input - 1, 1, 2)
(Test input - False, False, hr)

Error: template_path_and_file 'False' doesn't exist.
Troubleshooting tips:
        >Filenames must end with '.xlsx'
        >Check spelling 
        >Use forward slashes /'s in your path
        >Make sure you can access the file and/or path from your computer (Is it on your harddisk or the network(VPN)?
        >Call the Python Hotline (408)536.2086



(Test input - 1, 1, 3)
(Test input - False, False, min)

Error: template_path_and_file 'False' doesn't exist.
Troubleshooting tips:
        >Filenames must end wit

DataFrame written successfully to: '../data/sampOutput_28Jul2013.xlsx'

 




(Test input - 3, 1, 3)
(Test input - ../data/sampOutput.xlsx, False, min)

Template confirmed ../data/sampOutput.xlsx 
Save directory confirmed '../data/'
Template successfully saved to new file: '../data/sampOutput_28Jul201341.xlsx'
DataFrame written successfully to: '../data/sampOutput_28Jul201341.xlsx'

 




(Test input - 3, 1, 4)
(Test input - ../data/sampOutput.xlsx, False, sec)

Template confirmed ../data/sampOutput.xlsx 
Save directory confirmed '../data/'
Template successfully saved to new file: '../data/sampOutput_28Jul20134152.xlsx'
DataFrame written successfully to: '../data/sampOutput_28Jul20134152.xlsx'

 




(Test input - 3, 1, 5)
(Test input - ../data/sampOutput.xlsx, False, def_ts)

Template confirmed ../data/sampOutput.xlsx 
Save directory confirmed '../data/'
Template successfully saved to new file: '../data/sampOutput_28Jul20.xlsx'
DataFrame written successfully to: '../data/sampOutput_28

DataFrame written successfully to: '../data/sampOutput_.xlsx'

 




(Test input - 3, 4, 7)
(Test input - ../data/sampOutput.xlsx, ../data/, crunge)

Template confirmed ../data/sampOutput.xlsx 
Save directory confirmed '../data/'
Template successfully saved to new file: '../data/sampOutput_28Jul20.xlsx'
DataFrame written successfully to: '../data/sampOutput_28Jul20.xlsx'

 




(Test input - 3, 5, 1)
(Test input - ../data/sampOutput.xlsx, sampOutput2.xlsx, False)

Template confirmed ../data/sampOutput.xlsx 
Save directory confirmed '../data/'
Template successfully saved to new file: '../data/sampOutput2_28Jul20.xlsx'
DataFrame written successfully to: '../data/sampOutput2_28Jul20.xlsx'

 




(Test input - 3, 5, 2)
(Test input - ../data/sampOutput.xlsx, sampOutput2.xlsx, hr)

Template confirmed ../data/sampOutput.xlsx 
Save directory confirmed '../data/'
Template successfully saved to new file: '../data/sampOutput2_28Jul2013.xlsx'
DataFrame written successfully to: '../data/sampOutput2

# Scraps and junk from here on

In [ ]:
#*#*# for debug remove from final version
#*#*#    print('\n')
#*#*#    print(template_path_and_file_dict)
#*#*#    print(save_path_and_file_dict) 
#*#*#    print('\n')
#*#*#    print(template_path_and_file_dict)
#*#*#    print(save_path_and_file_dict)